# Explore the Nav2D environment

In [ ]:
import mujoco as mj
import gymnasium as gym
import nav2d
import pyautogui
import numpy as np

width = 400
height = 400
default_camera_config = {"azimuth" : 90, "elevation" : -90.0, "distance" : 3, "lookat" : [0.0, 0.0, 0.0]}

# Reference for setting visual flags https://mujoco.readthedocs.io/en/stable/APIreference/APItypes.html#mjtvisflag
visual_options = {2: True, 8: True}      # e.g., visualize the joints by setting mjVIS_JOINT (index 2) = True

# There are a few visualization things that cannot be set when making the env
# Ref - https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/mujoco/mujoco_rendering.py
# They can only be changed via keypresses in the gymnasium mujoco rendering. 
# Here are some flags to specify what key to press. The key presses are simulated using `pyautogui`
# (Super rough appproach but oh well)
DEFAULT_CAMERA = "overhead_camera"
ENABLE_FRAME = True                     # enable the body frames
RENDER_EVERY_FRAME = True              # similar sim speed as MuJoCo rendering when set to False, else slower

reward_scale= {
    "rew_head_scale": 10.0,
    "rew_head_approach_scale": 250.0,
    "rew_dist_scale": 05.0,
    "rew_dist_approach_scale": 190.0,
    "rew_time": -0.25,
    "rew_goal_scale": 5_000.0,
    "rew_obst_scale": -1_000.0
}
        
randomization_options = {
    "agent_freq": 1,
    "goal_freq": 1
}

env = gym.make("Nav2D-v0", 
               render_mode="human", 
               width=width, height=height,
               default_camera_config=default_camera_config,
               visual_options=visual_options,
               reward_scale_options=reward_scale,
               randomization_options=randomization_options
               )
obs, info = env.reset()

# Simulate keypress for visualization elements in gymnasium MuJoCo rendering
if DEFAULT_CAMERA=="overhead_camera": pyautogui.press('tab')
if ENABLE_FRAME: pyautogui.press('e') 
if not RENDER_EVERY_FRAME: pyautogui.press('d') 

obs_hist = []
obs_hist.append(obs)
for i in range(5):
    done = False
    # enable the body frame by simulating a keypress once lmao
    
    while not done:
        action = [0.0, 1.0]
        nobs, rew, term, trunc, info = env.step(action)

        delta_x, delta_y = nobs[:2]
        sin_theta, cos_theta = nobs[2:4]
        theta = (np.arctan2(sin_theta, cos_theta, dtype=np.float32) / np.pi * 180) % 360
        vel_x, vel_y, vel_z = nobs[4:7]
        lidar_scans = nobs[7:]
        # print(f"(d_x,d_y) = ({delta_x:4.3f},{delta_y:4.3f})  |   theta = {theta}   |   agent_vel = ({vel_x:4.3f},{vel_y:4.3f},{vel_z:4.3f})              ", end="\r")
        done = term or trunc
        obs = nobs if not done else env.reset()[0]
        # if done:
        #     print(f"eps: {env.unwrapped.episode_counter:4d} | goal bound: {env.unwrapped.goal_bound: 5.4f}")
        # if info:
        #     print(info)
        obs_hist.append(obs)
        # if done: print(nobs, info) 

env.close()

# Custom TD3

In [ ]:
import numpy as np
print(np.arange(0,1,0.25))
np.linspace(0,1,9)

## Library Import

In [ ]:
# --- import the custom-made TD3 algorithm
import gymnasium as gym
from gymnasium.wrappers import TimeLimit
import nav2d        # Have to import the nav2d Python script, else we can't make env
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

import json
import os, re
import sys
sys.path.insert(0,'..')
from algorithms import TD3

## Custom TD3 Training

In [ ]:
model_registry = {
    'TD3_v0': {
        'actor_config': [256, 256],
        'critic_config': [256, 256]
    },
}

MODEL_NAME = 'TD3_v0'
ALPHA1 = 1e-3
ALPHA2 = 1e-3
BETA = 1e-3
GAMMA = 0.99
TAU_C = 5e-3
TAU_A = 5e-3
SIGMA = 0.2
CLIP = 0.5

BUFFER_SIZE = 20_000
BUFFER_INIT = 2_000
BATCH_SIZE = 1024
  
UPDATE_FREQ = 2
UPDATE_STEP = 2
TRAIN_ITER = 50_000
RAND_GOAL_FREQ = 1
TRAIN_CRIT = {"pass_limit": 3, "pass_score": 0, 'coeff_var_limit': 1.0}
RESULT_FOLDER = 'Nav2D_TD3_results'
CUDA_ENABLED = True
EARLY_STOP = True

In [ ]:
# env = TimeLimit(gym.make("Nav2D-v0", render_mode="human"), max_episode_steps = 1_000)

env = gym.make("Nav2D-v0", render_mode=None, visual_options = {2: True, 8: True})
for i in range(1):    
    seed = np.random.randint(1,100)
    TD3_experiment = TD3(model_name = MODEL_NAME, model_registry=model_registry, env=env,
                     alpha1=ALPHA1,alpha2=ALPHA2,beta=BETA,gamma=GAMMA,
                     tau_c=TAU_C,tau_a=TAU_A,sigma=SIGMA,clip=CLIP,
                     buffer_size=BUFFER_SIZE,buffer_init=BUFFER_INIT, batch_size=BATCH_SIZE, 
                     update_f=UPDATE_FREQ, update_step=UPDATE_STEP, iter=TRAIN_ITER,
                     rand_goal_freq=RAND_GOAL_FREQ,
                     seed=seed,
                     train_crit=TRAIN_CRIT,
                     result_folder=RESULT_FOLDER,
                     cuda_enabled=CUDA_ENABLED)                 
    TD3_experiment.train(early_stop=EARLY_STOP,verbose=True)         

In [ ]:
type(env)

## Load and Simulate models

In [ ]:
def load_model(q_network: nn.Module, model_path):
    checkpoint = torch.load(model_path)
    q_network.load_state_dict(checkpoint['model_state_dict'])

def EMA_filter(reward: list, alpha: float=0.1):
        ''' Function that runs an exponential moving average filter along a datastream '''
        output = np.zeros(len(reward)+1)
        output[0] = reward[0]
        for idx, item in enumerate(reward):
            output[idx+1] = (1 - alpha) * output[idx] + alpha * item
        
        return output

def plot_fn(history, xlabel:str='step', ylabel: str='reward', alpha: float=0.0):
        ''' Function that plots the reward and filtered reward per episode, then saves the plot in a specified save directory'''
        n_episodes= len(history)
        episodes = range(n_episodes)
        filtered_reward_hist = EMA_filter(history, alpha)

        legend = []
        plt.figure(figsize=(20,6))
        plt.plot(episodes, history[:n_episodes], color = "blue"); legend.append(ylabel)
        if alpha:
            plt.plot(episodes, filtered_reward_hist[:n_episodes], color = "red"); legend.append('filtered '+ylabel)
        # plt.title(f'Total reward per episode - {self.hyperparam_config}')
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.legend(legend)
        plt.grid(which='both')

        plt.tight_layout()
        # if self.save_path:
        #     plt.savefig(os.path.join(self.save_path,'reward_history.png'))
        plt.show()

In [ ]:
class Actor(nn.Module):
    def __init__(self, obs_dim: int,
                 act_dim: int,
                 act_low, act_high, 
                 hidden_layers=[64,64], 
                 cuda_enabled=False):
        ''' Initialize the model and create a list of layers '''
        super().__init__()

        self.act_low, self.act_high = act_low, act_high

        self.layers = nn.ModuleList()
        self.obs_dim = obs_dim
        self.act_dim = act_dim
        # hidden layers
        for layer_size in hidden_layers:
            self.layers.append(nn.Linear(obs_dim, layer_size))
            obs_dim = layer_size
        # output layers
        self.layers.append(nn.Linear(obs_dim, self.act_dim))
        self.tanh = nn.Tanh()

        if cuda_enabled: self.cuda()

    def forward(self, input):
        for layer in self.layers[:-1]:
            input = torch.relu(layer(input))
        output = self.layers[-1](input)
        act_tanh = self.tanh(output)

        # rescale to environment bounds
        act = 0.5 * (act_tanh + 1.0) * (self.act_high - self.act_low) + self.act_low
        return act

In [ ]:
## Manually select a folder/run to load
run_number = 'run_00016'

## Find the paths to the param_config and model checkpoint
# RESULT_DIR = os.path.dirname(REINFORCE_experiment.save_path)    # Can run this after running one experiment
BASE_DIR = os.getcwd()
RESULT_DIR = os.path.join(BASE_DIR, RESULT_FOLDER)
RUN_DIR = os.path.join(RESULT_DIR, run_number)
MODEL_PATH = os.path.join(RUN_DIR,'q_network_checkpoint.pth')

## Find the model configuration
with open(os.path.join(RUN_DIR,"param_config.json"),'r') as f:
    data = json.load(f)
model_name = data['parameters']['model_name']
model_config = model_registry[model_name]['actor_config']

n_test_eps = 2

width = 1920
height = 1080
default_camera_config = {"azimuth" : 90, "elevation" : -90.0, "distance" : 3, "lookat" : [0.0, 0.0, 0.0]}
render_mode = 'human' if n_test_eps <= 5 else 'rgb_array'

env_test = gym.make('Nav2D-v0',
               render_mode=render_mode,
               width=width,height=height,
               default_camera_config=default_camera_config
            )

obs_space = env_test.observation_space.shape[0]
act_space = env_test.action_space.shape[0]
act_low = torch.as_tensor(env_test.action_space.low, dtype=torch.float32, device = 'cpu')
act_high = torch.as_tensor(env_test.action_space.high, dtype=torch.float32, device = 'cpu')

# Create the model according to the model version in the model registry
policy_net= Actor(obs_space, act_space, 
                    act_low=act_low, act_high=act_high, 
                    hidden_layers=model_config)
load_model(policy_net, MODEL_PATH)

act_hist = []
rew_hist = []
eps_rew_hist = []
torso_z_hist = []

noise_std = 0
truncated = True

for eps in range(n_test_eps):
    obs, _ = env_test.reset()
    done = False
    torso_z_hist.append(obs[0])

    while not done:
        with torch.no_grad():
            obs_t = torch.as_tensor(obs, dtype = torch.float32, device = 'cpu')
            act = policy_net(obs_t)
            act += torch.randn_like(act) * noise_std

            nobs, rew, term, trunc, _ = env_test.step(act.numpy())

            act_hist.append(act.numpy())
            torso_z_hist.append(obs[0])

            # rew_hist.append(rew)
            if truncated:
                rew_hist.append(rew)
                truncated = False
            else:
                rew_hist.append(rew_hist[-1]+rew)

            done = term or trunc
            if not done:
                obs = nobs  
            else:
                obs = env_test.reset()[0]
                truncated = True
                eps_rew_hist.append(rew_hist[-1])
                msg = 'done due to termination' if term else 'done due to truncation'
    # print(f'episode {eps:3d} ' + msg)
env_test.close()

plot_fn(rew_hist, ylabel='instantaneous reward')

# Statistical analysis
if n_test_eps > 1:
    print(f"Evaluation of {run_number} after {n_test_eps} eval episodes | µ = {mean(eps_rew_hist):5.2f} | σ = {stdev(eps_rew_hist):5.2f}")

else:
    # --- plot the z-position of the torso
    plot_fn(torso_z_hist, ylabel='torso z level')

    # --- unpack the actions and plot
    act1_hist, act2_hist, act3_hist, act4_hist, act5_hist, act6_hist = zip(*act_hist)
    legends=[]
    plt.figure(figsize=(20,6))
    plt.plot(range(len(act1_hist)), act1_hist, color = "blue"); legends.append(r"right thigh $\tau_{0}$")
    plt.plot(range(len(act2_hist)), act2_hist, color = "orange"); legends.append(r"right leg $\tau_{1}$")
    plt.plot(range(len(act3_hist)), act3_hist, color = "green"); legends.append(r"right foot $\tau_{2}$")
    plt.title("Right leg torques vs time")
    plt.legend(legends, bbox_to_anchor=[0.5, -0.1], loc='center', ncol=10)
    plt.grid(which='both')
    plt.show()

    legends=[]
    plt.figure(figsize=(20,6))
    plt.plot(range(len(act4_hist)), act4_hist, color = "blue"); legends.append(r"left thigh $\tau_{3}$")
    plt.plot(range(len(act5_hist)), act5_hist, color = "orange"); legends.append(r"left leg $\tau_{l2}$")
    plt.plot(range(len(act6_hist)), act6_hist, color = "green"); legends.append(r"left foot $\tau_{l3}$")
    plt.title("Left leg torques vs time")
    plt.legend(legends, bbox_to_anchor=[0.5, -0.1], loc='center', ncol=10)
    plt.grid(which='both')
    plt.show()

# SB3 TD3

## Library Import

In [ ]:
from stable_baselines3 import TD3,SAC
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env

import torch
import gymnasium as gym
from gymnasium.wrappers import RescaleAction
import nav2d        # Have to import the nav2d Python script, else we can't make env
import nav2d_testing
import numpy as np
import os, re, json, time
from datetime import datetime
from tqdm import tqdm

## Parameters & Functions

In [ ]:
def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Time taken to do evaluation - {end-start:5.3f}")

        return result
    return wrapper

In [ ]:
# this is the evaluation function:
# @timer_decorator
def eval_policy(env: gym.Env, 
         num_evals: int, 
         model):
    # reward list:
    eval_rew_hist = []

    # for each episode in the num_evals:
    for _ in range(num_evals):
        obs, _ = env.reset()
        done = False

        # initialize episodic reward:
        eval_rew = 0

        # while False:
        while not done:
            # get action and step:
            with torch.no_grad():
                action, _ = model.predict(obs, deterministic = True)
                nobs, reward, term, trunc, _ = env.step(action)
                done = term or trunc
                
                # advance reward:
                eval_rew += reward

                # advance observation, reset if not:
                obs = nobs if not done else env.reset()
    
        # append:
        eval_rew_hist.append(eval_rew)

    return np.mean(eval_rew_hist).round(3)

Environment parameters:

In [ ]:
reward_scale= {
    "rew_head_scale": 2.5,
    "rew_head_approach_scale": 50.0,
    "rew_dist_scale": 2.5,
    "rew_dist_approach_scale": 250.0,
    "rew_goal_scale": 2_000.0,
    "rew_obst_scale": -1_000.0
}

# vectorize environments
n_envs = 4

Policy parameters:

In [ ]:
learning_rate = 3e-4
buffer_size=int(1e6)
learning_starts=10_000
batch_size=2048 
tau=5e-3
gamma=0.99
train_freq=1
gradient_steps=1
action_noise=None
n_steps=1
policy_delay=2
target_policy_noise=0.1
target_noise_clip=0.2
verbose=0
tensor_board_log_dir="./results/Nav2D_TD3_SB3_tensorboard/"

pi_arch = [256, 256]
qf_arch = [256, 256]
policy_kwargs=dict(activation_fn=torch.nn.ReLU,
                   net_arch=dict(pi=pi_arch, qf=qf_arch))
use_custom_policy = False
vectorize = False
cuda_enabled = True

## SB3 Training

Create the environments for training and testing:

In [ ]:
if not vectorize:
    print("Making a single environment!")
    # # Make one environment using gymnasium API
    env = gym.make("Nav2D-v0", render_mode="human", max_episode_steps=1_000, reward_scale_options=reward_scale)
else:
    # stable_baselines3 API
    print("Making dummy vectorized environments!")
    env = make_vec_env("Nav2D-v0", 
                    n_envs=n_envs,
                    env_kwargs={"max_episode_steps": 1_000,
                                "reward_scale_options": reward_scale
                                },
                    vec_env_cls=DummyVecEnv)
        
# evaluation environment:
eval_env = gym.make("Nav2D-v0", max_episode_steps = 1_000, render_mode = "human", is_eval=True)

# # wrapper to rescale the action of the environment
# action_bounds = np.array([1.0, 0.0001, 1.0], dtype=np.float32)
# vec_env = RescaleAction(vec_env, min_action=-action_bounds, max_action=action_bounds)

Create the training model:

In [ ]:
# custom MlpPolicy

model = TD3("MlpPolicy", env, 
            learning_rate=learning_rate,         # lr for all networds - Q-values, Actor, Value function
            buffer_size=buffer_size,         # replay buffer size
            learning_starts=learning_starts,        # # of data collection step before training
            batch_size=batch_size,
            tau=tau,                   # polyak update coefficient
            gamma=gamma,
            train_freq=train_freq,
            gradient_steps=gradient_steps, 
            action_noise=action_noise, 
            n_steps=n_steps,                  # n-step TD learning
            policy_delay=policy_delay,             # the policy and target networks are updated every policy_delay steps
            target_policy_noise=target_policy_noise,    # stdev of noise added to target policy
            target_noise_clip=target_noise_clip,      # limit of asbsolute value of noise
            verbose=verbose,
            device="cuda" if cuda_enabled else "cpu",
            policy_kwargs=policy_kwargs if use_custom_policy else None,
            tensorboard_log=tensor_board_log_dir)

In [ ]:
# run parameters:
number_of_runs = 100
steps_per_run = 20_000
model_save_freq = int(number_of_runs / 20)

# model saving parameters:
base_path = os.path.join(os.getcwd(), "results", "Nav2D_TD3_SB3_results")
result_number = f"result_{len(os.listdir(base_path)):05d}"
results_path = os.path.join(base_path, result_number)

# using model.learn approach:
for run in tqdm(range(1,number_of_runs+1), ncols = 100, colour = "#33FF00", desc = "training progress"):
    # learn every run:
    model.learn(total_timesteps = steps_per_run, tb_log_name=f"{result_number}",reset_num_timesteps = False)
    # model.learn(total_timesteps = steps_per_run, reset_num_timesteps = False)

    # save a model once in a while
    if run % model_save_freq == 0:
        model.save(os.path.join(results_path, f"run_{run}"))

# save the last model
model.save(os.path.join(results_path, f"run_{run}"))

# close environment when done:
env.close()

# Save the result-params mapping into a json file
trial_to_param_path = os.path.join(base_path,'trial_to_param.json')
if os.path.exists(trial_to_param_path):
    with open(trial_to_param_path, "r") as f:
        data = json.load(f)
else:
    data = {result_number: ""}

hyperparam_codified = f"{learning_rate}_{buffer_size}_{learning_starts}_{batch_size}_{tau}_{gamma}_"
hyperparam_codified += f"{train_freq}_{gradient_steps}_{n_steps}_{policy_delay}_{target_policy_noise}_{target_noise_clip}_"
hyperparam_codified += f"{reward_scale['rew_head_scale']}_{reward_scale['rew_dist_scale']}_{reward_scale['rew_goal_scale']}_{reward_scale['rew_obst_scale']}"

timestamp = datetime.now().strftime("%y%m%d_%H%M")
hyperparam_codified_time = f"{timestamp}_" + hyperparam_codified

data[result_number] = hyperparam_codified_time

with open(trial_to_param_path, "w") as f:
    json.dump(data, f, indent=2)

# Optuna Studies

Create the Optuna study with the objective function 

In [ ]:
import optuna

def train_model(model,
                n_runs : int = 100,
                steps_per_run: int = 25_000,
                result_number: str | None = None,
                result_path: str | None = None):
    # model training parameters
    model_save_freq = int(n_runs / 10)

    # --- train the model
    for run in tqdm(range(1,n_runs+1), ncols = 100, colour = "#33FF00", desc = "training progress"):
        # learn every run:
        model.learn(total_timesteps = steps_per_run, tb_log_name=f"{result_number}",reset_num_timesteps = False)

        # save a model once in a while
        if run % model_save_freq == 0:
            model.save(os.path.join(result_path, f"run_{run}"))

## Optuna Study 1 - Reward scales

In [ ]:
def objective_rew_scale(trial):
    rew_head_scale = trial.suggest_float("rew_head_scale", low=0.0, high=5.0, step=0.5)
    rew_dist_scale = trial.suggest_float("rew_dist_scale", low=0.0, high=5.0, step=0.5)

    reward_scale= {
        "rew_head_scale": rew_head_scale,
        "rew_dist_scale": rew_dist_scale,
        "rew_goal_scale": 2_000,
        "rew_obst_scale": -100
    }

    # create the env
    env = make_vec_env("Nav2D-v0", 
                   n_envs=n_envs,
                   env_kwargs={
                       "max_episode_steps": 2_000,
                       "reward_scale_options": reward_scale
                   },
                   vec_env_cls=DummyVecEnv)
    
    # create the model
    model = TD3("MlpPolicy", env, 
            learning_rate=learning_rate,         # lr for all networds - Q-values, Actor, Value function
            buffer_size=buffer_size,         # replay buffer size
            learning_starts=learning_starts,        # # of data collection step before training
            batch_size=batch_size,
            tau=tau,                   # polyak update coefficient
            gamma=gamma,
            train_freq=train_freq,
            gradient_steps=gradient_steps, 
            action_noise=action_noise, 
            n_steps=n_steps,                  # n-step TD learning
            policy_delay=policy_delay,             # the policy and target networks are updated every policy_delay steps
            target_policy_noise=target_policy_noise,    # stdev of noise added to target policy
            target_noise_clip=target_noise_clip,      # limit of asbsolute value of noise
            verbose=verbose,
            policy_kwargs=policy_kwargs if use_custom_policy else None,
            tensorboard_log=tensor_board_log_dir)

    # model saving parameters:
    base_path = os.path.join(os.getcwd(), "results","Nav2D_TD3_SB3_results")
    result_number = f"result_{len(os.listdir(base_path)):05d}"
    result_path = os.path.join(base_path, result_number)

    # --- train the model
    train_model(model=model,
                n_runs=100, 
                steps_per_run=20_000,
                result_number=result_number,
                result_path=result_path)    
    env.close()

    # --- Save the result-params mapping into a json file
    trial_to_param_path = os.path.join(base_path,'trial_to_param.json')
    if os.path.exists(trial_to_param_path):
        with open(trial_to_param_path, "r") as f:
            data = json.load(f)
    else:
        data = {result_number: ""}

    hyperparam_codified = f"{learning_rate}_{buffer_size}_{learning_starts}_{batch_size}_{tau}_{gamma}_"
    hyperparam_codified += f"{train_freq}_{gradient_steps}_{n_steps}_{policy_delay}_{target_policy_noise}_{target_noise_clip}_"
    hyperparam_codified += f"{reward_scale['rew_head_scale']}_{reward_scale['rew_dist_scale']}_{reward_scale['rew_goal_scale']}_{reward_scale['rew_obst_scale']}"

    timestamp = datetime.now().strftime("%y%m%d_%H%M")
    hyperparam_codified_time = f"{timestamp}_" + hyperparam_codified

    data[result_number] = hyperparam_codified_time

    with open(trial_to_param_path, "w") as f:
        json.dump(data, f, indent=2)
    
    # --- Evaluate the model after training
    n_evals = 20
    eval_env = gym.make("Nav2D-v0", max_episode_steps = 1_000, render_mode = "rgb_array", is_eval=True)
    mean_eval_rew = eval_policy(env=eval_env, num_evals=n_evals, model=model)
    eval_env.close()
    
    return mean_eval_rew    # return the mean of evaluation reward as the maximizing objective

In [ ]:
study_name = "reward_scale_oct25_2"
study = optuna.create_study(storage=f"sqlite:///results/{study_name}.db", study_name=study_name, direction='maximize')
study.optimize(objective_rew_scale, n_trials=100)

In [ ]:
env.close()